## 네이버 영화 리뷰 감성분석
- 네이버 영화 리뷰 감성 분류하기(https://wikidocs.net/44249) 참조
    - 앞에 전처리까지는 동일하나, 뒤에부터는 딥러닝임으로 전처리까지만 참고

In [1]:
import pandas as pd
train_df = pd.read_csv("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", sep='\t')
test_df = pd.read_csv("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", sep='\t')

In [2]:
train_df.shape, test_df.shape

((150000, 3), (50000, 3))

In [3]:
train_df.head(3)

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


#### 1. 데이터 전처리

- train data

In [6]:
# 결측치 확인
train_df.isna().sum()

id          0
document    5
label       0
dtype: int64

In [7]:
# 결측치 데이터 삭제
train_df.dropna(how='any', inplace=True)
train_df.shape

(149995, 3)

In [8]:
# 중복 데이터 확인
train_df.document.nunique()     # 고유한게 몇개있나?

146182

In [9]:
# 중복 데이터 제거
train_df.drop_duplicates(subset=['document'], inplace=True)
train_df.shape

(146182, 3)

In [11]:
# 데이터 분포
train_df.label.value_counts()

# 0,1이 비슷한 비율로 있음 => 좋은 데이터셋이다!

0    73342
1    72840
Name: label, dtype: int64

- test data

In [12]:
# 결측치 제거, 중복데이터 제거
test_df.dropna(how='any', inplace=True)
test_df.drop_duplicates(subset=['document'], inplace=True)
test_df.shape

(49157, 3)

In [13]:
# 분포확인
test_df.label.value_counts()

1    24711
0    24446
Name: label, dtype: int64

#### 2. 텍스트 전처리

- train data

In [15]:
# 한글 이외의 문자는 공백으로 처리하고 strip
train_df.document = train_df.document.str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣]', ' ', regex=True).str.strip()
# regex=True: 트루로 해줘야 바뀜
# str메소드에 replace, strip이 있음
train_df.head(3)

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠 포스터보고 초딩영화줄 오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


In [16]:
# 한글이 없는 글 --> ''만 남게됨
# ''만 남은 데이터는 제거: np.nan으로 대체 후 dropna 실행
import numpy as np
train_df.document.replace('', np.nan, inplace=True)
train_df.document.isna().sum()

789

In [18]:
train_df.dropna(how='any', inplace=True)
train_df.shape

(145393, 3)

- test data

In [19]:
test_df.document = test_df.document.str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣]', ' ', regex=True).str.strip()
test_df.document.replace('', np.nan, inplace=True)
test_df.dropna(how='any', inplace=True)
test_df.shape

(48852, 3)

In [20]:
train_df.to_csv('data/네이버영화리뷰 train 전처리완료.tsv', sep='\t', index=False)
test_df.to_csv('data/네이버영화리뷰 test 전처리완료.tsv', sep='\t', index=False)

#### 3.한글 처리

In [21]:
from konlpy.tag import Okt
okt = Okt()


In [35]:
text = '흠 포스터보고 초딩영화줄 오버연기조차 가볍지 않구나'
okt.morphs(text)

['흠', '포스터', '보고', '초딩', '영화', '줄', '오버', '연기', '조차', '가볍지', '않구나']

In [36]:
okt.morphs(text, stem=True)

['흠', '포스터', '보고', '초딩', '영화', '줄', '오버', '연기', '조차', '가볍다', '않다']

In [37]:
with open('data/한글불용어100.txt') as st:
    lines = st.readlines()
print(lines[0])

이	VCP	0.018279601



In [38]:
stop_words = []
for line in lines:
    word = line.split('\t')[0]
    stop_words.append(word)

In [39]:
# 위 코드와 동일한 코드식
stop_words = [line.split('\t')[0] for line in lines]


In [40]:
stop_words[:10]

['이', '있', '하', '것', '들', '그', '되', '수', '이', '보']

In [41]:
# 형태소 분석기를 돌려서 나온 결과에서 불용어 제거하기
morphs = okt.morphs(text, stem=True)
clean_morphs = [morph for morph in morphs if morph not in stop_words]
clean_morphs


['흠', '포스터', '보고', '초딩', '영화', '줄', '오버', '연기', '조차', '가볍다', '않다']

In [42]:
# Feature 변환을 위한 Vectorizer의 입력은 리스트가 아닌 문자열이어야 함
clean_text = ' '.join(clean_morphs)
clean_text

'흠 포스터 보고 초딩 영화 줄 오버 연기 조차 가볍다 않다'

- train/test dataset에 적용


In [ ]:
from tqdm import tqdm

X_train = []
for review in tqdm(train_df.document):
    morphs = okt.morphs(review, stem=True)
    clean_morph_review = ' '.join([morph for morph in morphs if morph not in stop_words])
    X_train.append(clean_morph_review)